In [4]:
import sqlite3
import pandas as pd
import numpy as np
import warnings
import re
import nltk
import sqlite3
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
warnings.filterwarnings("ignore")
from textblob import Word
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
import random
import requests
from io import BytesIO
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to /home/mine44/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mine44/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/mine44/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
#Load dataset
train = pd.read_csv('imdb_movies_cleaned.csv',sep='|')
test = pd.read_csv('imdb_marvel_cleaned.csv',sep='|')

In [3]:
train.head()

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres,overview,release_date,tagline,averageRating,numVotes
0,tt0015400,movie,The Thief of Bagdad,1924,155,"Adventure,Family,Fantasy",recalcitrant thief vies duplicitous mongol rul...,3/18/24,happiness must earned,7.7,6324
1,tt0019563,movie,West of Zanzibar,1928,65,"Drama,Mystery",silent film musical score vengefulness cuckold...,11/24/28,NaN,7.2,1817
2,tt0022958,movie,Grand Hotel,1932,112,"Drama,Romance",guest posh berlin hotel struggle worry scandal...,5/25/32,thank star great entertainment,7.3,19576
3,tt0024216,movie,King Kong,1933,100,"Adventure,Horror,Sci-Fi",adventure film film crew search monster remote...,4/7/33,monster creation dawn break loose world today,7.9,86102
4,tt0027977,movie,Modern Times,1936,87,"Comedy,Drama,Romance",tramp struggle live modern industrial society ...,2/5/36,stand alone greatest entertainer modern time o...,8.5,242411


In [4]:
test.head()

,primaryTitle,runtimeMinutes,genres,overview,release_date,tagline,averageRating,numVotes
0,Black Panther: Wakanda Forever,161,"Action,Adventure,Drama",people wakanda fight protect home intervening ...,"November 11, 2022",forever,7.3,74000
1,Black Panther,134,"Action,Adventure,Sci-Fi",challa heir hidden advanced kingdom wakanda mu...,"February 16, 2018",long live king,7.3,762000
2,Thor: Love and Thunder,118,"Action,Adventure,Comedy",thor enlists help valkyrie korg ex girlfriend ...,"July 8, 2022",every god plan,6.3,309000
3,She-Hulk: Attorney at Law,35,"Action,Adventure,Comedy",jennifer walter navigates complicated life sin...,"August 18, 2022",like angry,5.2,159000
4,Spider-Man: No Way Home,148,"Action,Adventure,Fantasy",spider man identity revealed peter asks doctor...,"December 17, 2021",multiverse unleashed,8.3,744000


- Data preparation

In [5]:
#Remove movies of train data if they exist in the test data
train = train[~train['primaryTitle'].isin(test['primaryTitle'])].reset_index(drop=True)

In [6]:
#Combine the overview and tagline of movies
train['overview'] = train['overview'].fillna('')
train['tagline'] = train['tagline'].fillna('')
test['overview'] = test['overview'].fillna('')
test['tagline'] = test['tagline'].fillna('')
train['content'] = train['overview'] + ' ' + train['tagline']
test['content'] = test['overview'] +' ' + test['tagline']

In [7]:
#Download Google Pretrained Word2Vec model
# import gensim.downloader as api
# word2vec = api.load('word2vec-google-news-300')
# word2vec.save_word2vec_format('word2vec-google-news-300.bin', binary=True)
word2vec = KeyedVectors.load_word2vec_format('word2vec-google-news-300.bin', binary=True)

In [8]:
def mapping_text_w2v(text):
    word_list = [word for word in text.split() if word in word2vec]
    text_vector = np.zeros(300)
    for word in word_list:
        text_vector += word2vec[word]
    avg_text_vector = text_vector/len(word_list) 
    
    return avg_text_vector

In [11]:
#Apply word2vec model to content of data
train_w2v = pd.DataFrame(train['content'].apply(lambda x:mapping_text_w2v(x)).to_list(),index=train['primaryTitle'].values).dropna()
test_w2v = pd.DataFrame(test['content'].apply(lambda x:mapping_text_w2v(x)).to_list(),index=test['primaryTitle'].values).dropna()

- Model building

In [12]:
cosine_df = pd.DataFrame(cosine_similarity(train_w2v,test_w2v),columns=test_w2v.index,index=train_w2v.index)
cosine_df

,Black Panther: Wakanda Forever,Black Panther,Thor: Love and Thunder,She-Hulk: Attorney at Law,Spider-Man: No Way Home,Avengers: Endgame,Doctor Strange in the Multiverse of Madness,Eternals,Avengers: Infinity War,Iron Man,...,Doctor Strange,Ms. Marvel,Iron Man 2,What If...?,Ant-Man,Spider-Man: Far from Home,Captain America: The Winter Soldier,The Incredible Hulk,Iron Man 3,The Falcon and the Winter Soldier
The Thief of Bagdad,0.465942,0.571657,0.582132,0.512997,0.525446,0.497744,0.553251,0.470980,0.559724,0.537286,...,0.482147,0.529655,0.609875,0.380780,0.562641,0.537133,0.592528,0.524214,0.452625,0.508313
West of Zanzibar,0.451138,0.376179,0.534077,0.467043,0.494663,0.500388,0.510856,0.445950,0.480849,0.477417,...,0.458228,0.504807,0.565625,0.389162,0.528984,0.447807,0.563333,0.473547,0.447483,0.515677
Grand Hotel,0.472108,0.423655,0.438778,0.415755,0.456980,0.374454,0.515664,0.411960,0.456031,0.384730,...,0.408396,0.526504,0.504658,0.356868,0.487536,0.420368,0.516828,0.426042,0.469217,0.391345
King Kong,0.527217,0.585895,0.604339,0.585460,0.623410,0.576722,0.660966,0.584099,0.601909,0.567365,...,0.576265,0.637507,0.597039,0.618181,0.667782,0.627108,0.619287,0.648423,0.541912,0.633927
Modern Times,0.653682,0.615866,0.618740,0.582095,0.643313,0.563562,0.664378,0.638755,0.632962,0.569247,...,0.663663,0.730943,0.717762,0.578036,0.679272,0.619469,0.644279,0.584853,0.584132,0.517404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Better Angels,0.670158,0.666125,0.697002,0.664376,0.684932,0.580292,0.735084,0.650552,0.623639,0.621089,...,0.660757,0.713367,0.760703,0.527916,0.698198,0.661141,0.741467,0.631486,0.608274,0.645482
The Conjuring 2,0.530304,0.557379,0.614626,0.617031,0.634285,0.496799,0.678902,0.491038,0.505984,0.545981,...,0.477822,0.556116,0.606286,0.430963,0.628447,0.532356,0.600428,0.605723,0.528905,0.591814
Love & Friendship,0.506328,0.492168,0.706455,0.605691,0.670633,0.469038,0.642900,0.507550,0.475994,0.578914,...,0.483012,0.587470,0.668312,0.347544,0.645286,0.533532,0.667323,0.594553,0.496924,0.634054
Morgan,0.439212,0.541280,0.426203,0.438860,0.463260,0.436442,0.488962,0.368261,0.458868,0.415369,...,0.423020,0.409409,0.494277,0.423393,0.460912,0.510094,0.414724,0.498687,0.403922,0.372404


In [13]:
#Test case
cosine_df['Avengers: Endgame'].sort_values(ascending=False).head(10)

Thor: The Dark World             0.782388
Masters of the Universe          0.763835
Riddick                          0.755285
Dragon Ball Z: Battle of Gods    0.755244
Mortal Kombat: Annihilation      0.754360
The Chronicles of Riddick        0.752642
Hellboy II: The Golden Army      0.748139
Mortal Kombat                    0.747714
Men in Black II                  0.744225
Green Lantern                    0.739300
Name: Avengers: Endgame, dtype: float64

In [14]:
#Test case
cosine_df['Spider-Man: No Way Home'].sort_values(ascending=False).head(10)

The Amazing Spider-Man        0.828553
The Amazing Spider-Man 2      0.812536
Tales from Earthsea           0.808512
Serena                        0.797538
Jumanji                       0.794187
Ragnarok                      0.791178
Naan Kadavul                  0.786898
Silent Hill: Revelation       0.786127
Resident Evil: Retribution    0.784893
The Smurfs 2                  0.784717
Name: Spider-Man: No Way Home, dtype: float64

In [15]:
cosine_df.to_csv('movies_recommendation.csv',sep='|')